<a href="https://colab.research.google.com/github/janaat22/NLP_NER/blob/master/NER_Roadster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')


In [114]:
pip install sklearn-crfsuite


In [115]:
pip install nltk

In [0]:
import nltk

In [117]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [118]:
nltk.download('gazetteers')

[nltk_data] Downloading package gazetteers to /root/nltk_data...
[nltk_data]   Package gazetteers is already up-to-date!


True

In [119]:
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [0]:
from itertools import chain


import sklearn
import scipy.stats
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#from sklearn_crfsuite import CRF, scorers, metrics
import sklearn_crfsuite
from sklearn_crfsuite import CRF
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer

from nltk.stem.porter import *
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from nltk.corpus import names, ieer, gazetteers

In [0]:
locations = gazetteers.words()
locations = [x.lower() for x in locations]
names_list = names.words()
names_list = [x.lower() for x in names_list]

In [122]:
word = 'LA'
word.lower() in locations

True

In [123]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:


def file_to_list(file_name):  
  training_key_value = []
  word_dict = {}
  tags = {}
  word_list = []
  sentence = 1
  file1 = open(file_name, 'r')
  Lines = file1.readlines() 
  for line in Lines:    
    training_key_value = line.split()
    if(len(training_key_value) == 2):
      word = training_key_value[0]
      tag = training_key_value[1]
      word_dict[word] = tag
      tokenized = nltk.word_tokenize(word)
      tagged = nltk.pos_tag(tokenized)
      word_list.append(([sentence, word, tagged[0][1], tag])) 
      tags.update(tagged)
    else:
      sentence = sentence + 1
    file1.close()
  return word_dict,word_list

#Train data
word_dict_train, word_list_train = file_to_list('train.txt')
# print(word_dict_train.keys())
# array_of_words_train = np.array(list(word_dict_train.keys()))
# array_of_values_train = np.array(list(word_dict_train.values()))
# X_Y_train = np.column_stack((array_of_words_train, array_of_values_train))

#Dev data
word_dict_dev,word_list_dev = file_to_list('dev.txt')
# array_of_words_dev = np.array(list(word_dict_dev.keys()))
# array_of_values_dev = np.array(list(word_dict_dev.values()))
# X_Y_dev = np.column_stack((array_of_words_dev, array_of_values_dev))




In [0]:
def file_to_list_test(file_name):  
  training_key_value = []
  word_dict = {}  
  word_list = []
  training_key_value = [] 
  test_word_list = []
  file1 = open(file_name, 'r')
  sentence_num = 1
  Lines = file1.readlines() 
  # print("print" + str(len(Lines[11].split('\n'))))
  for line in Lines:    
    if(len(line) > 1):      
      word = line.replace('\n', '')
      tokenized = nltk.word_tokenize(word)
      tagged = nltk.pos_tag(tokenized)
      # print(sentence_num)
      test_word_list.append(([sentence_num, word, tagged[0][1]]))
    else:
      # print("\nelse")
      sentence_num = sentence_num + 1
  file1.close()
  return test_word_list



#Test data
word_list_test = file_to_list_test('test_no_tag.txt')


In [126]:
train_data = pd.DataFrame(word_list_train)
print(train_data.head(n = 4))

dev_data = pd.DataFrame(word_list_dev)
print(dev_data.head(n = 4))


test_data = pd.DataFrame(word_list_test)
print(test_data.head(n = 10))
# print(word_list_test)

   0                1    2  3
0  1  @SammieLynnsMom   NN  O
1  1         @tg10781   NN  O
2  1             they  PRP  O
3  1             will   MD  O
   0       1   2  3
0  1    STOP  NN  O
1  1    WHAT  WP  O
2  1  YOU'RE  NN  O
3  1   DOING  NN  O
   0         1    2
0  1        We  PRP
1  1       're  VBP
2  1    slaves  NNS
3  1        to   TO
4  1         a   DT
5  1     world   NN
6  1        we  PRP
7  1  mastered  VBN
8  1         .    .
9  2         I  PRP


In [127]:
# train_data.columns = train_data.iloc[1]
# train_data = train_data[1:]
train_data.columns = ['Sentence','Word','POS','Tag']
# train_data = train_data.reset_index(drop=True)
train_data.head(n=5)

# dev_data.columns = dev_data.iloc[1]
# dev_data = dev_data[1:]
dev_data.columns = ['Sentence','Word','POS','Tag']
# dev_data = dev_data.reset_index(drop=True)
dev_data.head(n=5)

# test_data.columns = test_data.iloc[0]
# test_data = test_data[0:]
test_data.columns = ['Sentence','Word','POS']
# test_data = test_data.reset_index(drop=True)
test_data.head(n= 10)


,Sentence,Word,POS
0,1,We,PRP
1,1,'re,VBP
2,1,slaves,NNS
3,1,to,TO
4,1,a,DT
5,1,world,NN
6,1,we,PRP
7,1,mastered,VBN
8,1,.,.
9,2,I,PRP


In [128]:
test_data.shape

(42413, 3)

In [0]:
# A class to retrieve the sentences from the dataset
class getsentence(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

class getsentence_ForTest(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p) for w, p in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist())
                                                      ]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [130]:
train_getter = getsentence(train_data)
train_sentences = train_getter.sentences

dev_getter = getsentence(dev_data)
dev_sentences = dev_getter.sentences

test_getter = getsentence_ForTest(test_data)
test_sentences = test_getter.sentences

#This is how a sentence will look like. 
print(train_sentences[0])

[('@SammieLynnsMom', 'NN', 'O'), ('@tg10781', 'NN', 'O'), ('they', 'PRP', 'O'), ('will', 'MD', 'O'), ('be', 'VB', 'O'), ('all', 'DT', 'O'), ('done', 'VBN', 'O'), ('by', 'IN', 'O'), ('Sunday', 'NNP', 'O'), ('trust', 'NN', 'O'), ('me', 'PRP', 'O'), ('*wink*', 'NN', 'O')]


In [0]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
# Feature set
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        # 'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'inLocations': word.lower() in locations,
        'inNames' : word.lower() in names_list,
        'isTag' : word.find('@'),
        'hasCapitalLetter' : word[0].isupper(),
        'stem' : stemmer.stem(word),        
        # 'lemma' : lemmatizer.lemmatize(word),
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:inLocations': word1.lower() in locations,
            '-1:inNames' : word1.lower() in names_list,
            '-1:hasCapitalLetter' : word1[0].isupper(),
            '-1:stem' : stemmer.stem(word1),            
            # '-1:lemma' : lemmatizer.lemmatize(word1),
             
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            # '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:inLocations': word1.lower() in locations,
            '+1:inNames' : word1.lower() in names_list,
            '+1:hasCapitalLetter' : word1[0].isupper(),
            '+1:stem' : stemmer.stem(word1),            
            # '+1:lemma' : lemmatizer.lemmatize(word),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [132]:
print(test_sentences[5])
sent2features(train_sentences[3])[0]
# sent2features(test_sentences[7])[0]

[('Happy', 'JJ'), ('v-day', 'NN'), ('😌', 'NN'), ('❤', 'NN'), ('️', 'NN'), ('😂😂😂😂', 'NN'), ('#noshame', '#')]


{'+1:hasCapitalLetter': False,
 '+1:inLocations': False,
 '+1:inNames': False,
 '+1:postag': 'RB',
 '+1:postag[:2]': 'RB',
 '+1:stem': 'perhap',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 'BOS': True,
 'bias': 1.0,
 'hasCapitalLetter': False,
 'inLocations': False,
 'inNames': False,
 'isTag': 0,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'stem': '@ls_n',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': False,
 'word[-2:]': '_n',
 'word[-3:]': 's_n'}

In [0]:
#Creating the train and test set

X_train = [sent2features(s) for s in train_sentences]
y_train = [sent2labels(s) for s in train_sentences]

X_dev = [sent2features(s) for s in dev_sentences]
y_dev = [sent2labels(s) for s in dev_sentences]

X_test = [sent2features(s) for s in test_sentences]


In [0]:
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [0]:
crf = crf.fit(X_train,y_train)

In [136]:
pred = cross_val_predict(estimator=crf, X = X_train,y = y_train, cv=10)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [137]:
#evaluate the model
report = flat_classification_report(y_pred=pred, y_true=y_train)
print(report)
# f1 = metrics.flat_f1_score(y_train, y_pred, average='weighted', labels=labels)
# print("F1 weighted avg: " + str(f1))

              precision    recall  f1-score   support

   B-company       0.88      0.47      0.61       171
     B-group       0.51      0.18      0.27       106
  B-location       0.77      0.54      0.63       380
     B-other       0.55      0.27      0.36       225
    B-person       0.79      0.63      0.70       449
   B-product       0.74      0.21      0.32        97
     B-title       0.56      0.22      0.32        68
   I-company       0.67      0.11      0.19        36
     I-group       0.34      0.13      0.19        84
  I-location       0.58      0.34      0.43       154
     I-other       0.50      0.30      0.37       320
    I-person       0.79      0.70      0.74       215
   I-product       0.65      0.19      0.29        80
     I-title       0.45      0.19      0.27        77
           O       0.97      0.99      0.98     44007

    accuracy                           0.96     46469
   macro avg       0.65      0.36      0.45     46469
weighted avg       0.96   

In [138]:
# labels = list(crf.classes_)

# labels.remove('O')
print(labels)


['B-location', 'I-location', 'B-title', 'I-title', 'B-company', 'B-product', 'B-person', 'B-other', 'I-other', 'B-group', 'I-group', 'I-product', 'I-company', 'I-person']


In [139]:
y_pred = crf.predict(X_dev)
metrics.flat_f1_score(y_dev, y_pred,
                      average='weighted', labels=labels)

0.3044540443631359

In [140]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

accuracy = metrics.flat_accuracy_score(y_dev, y_pred)
print("Accuracy: " + str(accuracy))
f1 = metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels)
precision = metrics.flat_precision_score(y_dev, y_pred, average='weighted', labels=labels)
print("Precision : "  + str(precision))

recall = metrics.flat_recall_score(y_dev, y_pred, average='weighted', labels=labels)
print("Recall : " + str(recall))
print("F1 weighted avg: " + str(f1))

              precision    recall  f1-score   support

   B-company      0.500     0.205     0.291        39
   I-company      0.000     0.000     0.000        10
     B-group      0.273     0.027     0.049       111
     I-group      0.000     0.000     0.000        48
  B-location      0.609     0.455     0.520       154
  I-location      0.556     0.370     0.444        81
     B-other      0.383     0.136     0.201       132
     I-other      0.211     0.237     0.223        97
    B-person      0.621     0.509     0.559       171
    I-person      0.600     0.632     0.615        95
   B-product      0.250     0.027     0.049        37
   I-product      0.333     0.017     0.031       121
     B-title      0.125     0.059     0.080        17
     I-title      0.091     0.067     0.077        15

   micro avg      0.482     0.270     0.346      1128
   macro avg      0.325     0.196     0.224      1128
weighted avg      0.422     0.270     0.304      1128

Accuracy: 0.9415780087325

# Hyper Parameter Optimization

In [141]:
Model_X = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 34.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=False, averaging=None,
                                 c=None, c1=0.1, c2=0.1,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_t...
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f0633ea00f0>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False,
                   scoring=make_scorer(flat_f1_score, average=weighted, l

# Best Parameters

In [142]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.12873556177131187, 'c2': 0.003090506667659071}
best CV score: 0.46966321802772804
model size: 0.50M


In [143]:
crf = rs.best_estimator_
y_pred = crf.predict(X_dev)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

accuracy = metrics.flat_accuracy_score(y_dev, y_pred)
print("accuracy :" + str(accuracy))
precision = metrics.flat_precision_score(y_dev, y_pred, average='weighted', labels=labels)
print("Precision : "  + str(precision))
recall = metrics.flat_recall_score(y_dev, y_pred, average='weighted', labels=labels)
print("Recall : " + str(recall))
f1 = metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels)
print("F1 weighted avg: " + str(f1))

              precision    recall  f1-score   support

   B-company      0.615     0.205     0.308        39
   I-company      0.000     0.000     0.000        10
     B-group      0.250     0.027     0.049       111
     I-group      0.000     0.000     0.000        48
  B-location      0.619     0.455     0.524       154
  I-location      0.614     0.432     0.507        81
     B-other      0.438     0.159     0.233       132
     I-other      0.218     0.278     0.244        97
    B-person      0.569     0.532     0.550       171
    I-person      0.550     0.632     0.588        95
   B-product      0.750     0.081     0.146        37
   I-product      0.667     0.017     0.032       121
     B-title      0.000     0.000     0.000        17
     I-title      0.000     0.000     0.000        15

   micro avg      0.479     0.284     0.356      1128
   macro avg      0.378     0.201     0.227      1128
weighted avg      0.473     0.284     0.313      1128

accuracy :0.9405325625730

In [144]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-location -> I-location 8.617318
B-title -> I-title 8.245135
B-person -> I-person 8.192486
B-group -> I-group 8.142497
B-product -> I-product 7.960793
B-other -> I-other 7.778814
I-title -> I-title 7.712788
I-product -> I-product 7.283203
I-other -> I-other 6.837604
I-group -> I-group 6.794987
I-location -> I-location 6.623692
B-company -> I-company 6.422857
I-company -> I-company 5.723954
I-person -> I-person 4.821123
O      -> O       4.129329
O      -> B-person 2.577534
O      -> B-product 2.227587
O      -> B-company 2.205858
O      -> B-title 2.012863
O      -> B-group 1.936574

Top unlikely transitions:
B-company -> O       0.212714
I-person -> O       0.061693
I-other -> O       0.059648
B-location -> O       0.056948
I-product -> O       0.021211
B-location -> B-other 0.000630
I-company -> B-location 0.000225
I-location -> B-group 0.000022
I-title -> O       -0.000079
B-product -> O       -0.056850
B-other -> O       -0.066541
B-group -> O       -0.0791

In [145]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
13.169553 B-company stem:twitter
12.260513 B-person stem:pope
10.855152 B-product stem:iphon
9.791072 B-company stem:facebook
9.626040 B-product stem:ipod
9.445102 B-person stem:4dbling
8.479214 B-product +1:stem:devic
8.332874 B-location stem:jupit
8.022778 B-other  stem:dynamit
7.813562 B-group  stem:#padr
7.499507 B-company stem:walmart
7.483995 B-title  stem:eurovis
7.342285 B-product stem:pringl
7.337845 B-location +1:stem:mansion
7.257145 B-other  word[-2:]:BL
7.187339 B-company stem:playboy
7.145666 B-person stem:bieber
7.102043 B-title  stem:icarli
7.061583 B-other  stem:thanksgiv
6.976447 B-product stem:xbox
6.785169 B-other  stem:rockhous
6.363909 B-other  stem:treasuri
5.935536 B-other  stem:dem
5.925091 B-company stem:yahoo
5.795075 B-person stem:snoop
5.774077 B-company stem:reuter
5.743445 B-group  +1:stem:gear
5.737556 B-location stem:kilkenni
5.724298 B-location word[-3:]:ham
5.705277 B-person stem:beyonc

Top negative:
-2.389014 O        stem:rose
-2.4018

In [146]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(y_pred[0][0])
print(X_test[0][0])
print(len(X_test))
print(len(y_pred))

O
{'bias': 1.0, 'word[-3:]': 'We', 'word[-2:]': 'We', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'PRP', 'postag[:2]': 'PR', 'inLocations': False, 'inNames': False, 'isTag': -1, 'hasCapitalLetter': True, 'stem': 'We', 'BOS': True, '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'VBP', '+1:postag[:2]': 'VB', '+1:inLocations': False, '+1:inNames': False, '+1:hasCapitalLetter': False, '+1:stem': "'re"}
4270
4270


In [0]:
outR = open("test_no_tag.txt", "r")
outF = open("result.txt", "w")
lines = outR.readlines()
pred_line = 0
pred_word = 0
for line in lines:
  if(len(line)>1):
    outF.write(line.replace('\n','')+"\t\t"+y_pred[pred_line][pred_word]+"\n")
    pred_word = pred_word + 1
  else:
    outF.write("\n")
    pred_word = 0
    pred_line = pred_line + 1


outR.close()
outF.close()